# Prediction Algorithms

## Preliminaries

In [97]:
import os
import sys
import networkx as nx
import numpy as np

from os.path import dirname
from networkx.drawing.nx_pydot import to_pydot

note_dir = os.getcwd()

root_dir = dirname(note_dir)
src_dir = os.path.join(root_dir, "src")

sys.path.append(src_dir)

import morpheus
from morpheus.utils.encoding import code_to_query
from morpheus.algo.selection import base_selection_algorithm
from morpheus.tests import default_chain, default_ensemble, default_dataset, default_m_list_for_mercs
from morpheus.graph import model_to_graph

In [98]:
def get_ids(g, kind='desc'):
    
    if kind in {'s', 'src', 'source', 'd', 'desc', 'descriptive'}:
        r = [g.nodes()[n]['idx'] for n, in_degree in g.in_degree
             if in_degree == 0
             if g.nodes()[n]['kind'] == 'data']
        
    elif kind in {'t', 'tgt', 'targ', 'target'}:
        r = [g.nodes()[n]['idx'] for n, in_degree in g.out_degree
             if out_degree == 0
             if g.nodes()[n]['kind'] == 'data']
    else:
        msg = """
        Did not recognize kind:   {}
        """.format(kind)
        raise ValueError(msg)
    
    return set(r)

## Collect Basic Building Blocks

In [7]:
X, _ = default_dataset()
X = X.values
m_list = default_m_list_for_mercs(X)


        Learning model with desc ids:    [0, 1, 2, 4, 5, 6, 7]
                            targ ids:    [3]
        

        Learning model with desc ids:    [0, 1, 3, 4, 5, 6, 7]
                            targ ids:    [2]
        

        Learning model with desc ids:    [0, 1, 2, 3, 4, 5, 7]
                            targ ids:    [6]
        
Unknown label type: 'continuous'

        Learning model with desc ids:    [0, 1, 2, 3, 5, 6, 7]
                            targ ids:    [4]
        

        Learning model with desc ids:    [0, 2, 3, 4, 5, 6, 7]
                            targ ids:    [1]
        

        Learning model with desc ids:    [0, 1, 2, 3, 4, 5, 6]
                            targ ids:    [7]
        

        Learning model with desc ids:    [0, 1, 2, 3, 4, 6, 7]
                            targ ids:    [5]
        

        Learning model with desc ids:    [1, 2, 3, 4, 5, 6, 7]
                            targ ids:    [0]
        


In [8]:
g_list = [model_to_graph(m) for m in m_list]

In [96]:
nx.DiGraph()

## IT

In [94]:
def it_algorithm(g_list, q_code, max_steps=4):
    
    # Init
    q_desc, q_targ, q_miss = code_to_query(q_code)
    
    avl_desc = set(q_desc)
    avl_targ = set(q_targ + q_miss)
    
    avl_grph = g_list
    
    g_res = nx.DiGraph()
    
    for step in range(max_steps):
        
        q_code = query_to_code(avl_desc, avl_targ, [])
        
        
        g_nxt = mrai_algorithm(avl_grph, q_code, complete= (max_step - step == 1))
        g_res = nx.compose(g_res, g_next)
        
        g_nxt_targ = get_ids(g_nxt, kind='targ')
        
        g_nxt_mods = set([n for n in g.nodes()
                          if g.nodes()[n]['kind'] == 'model'])
        
        avl_desc = avl_desc.union(g_nxt_targ)
        avl_targ = avl_targ.difference(g_nxt_targ)
        
        avl_grph = [g for g in avl_graph
                    if len(g_nxt_mods.intersection(set(g))) > 0]
        
        
    g_res = prune(g_res)
    
    return g_res

## RW-algorithm

In [ ]:
def rw_algorithm(g_list, q_code, max_chain_size=5):
    
    # Init
    q_desc, q_targ, q_miss = code_to_query(q_code)
    
    avl_desc = set(q_desc)
    avl_targ = set(q_targ)
    
    avl_grph = g_list
    
    g_res = nx.DiGraph()
    
    for step in range(max_chain_size):
        
        q_code = query_to_code(avl_desc, avl_targ, [])
        
        g_nxt = mrai_algorithm(avl_grph, q_code, stochastic=True)
        g_res = nx.compose(g_res, g_next)
        
        g_nxt_mods = set([n for n in g.nodes()
                          if g.nodes()[n]['kind'] == 'model'])
        g_res_desc, g_res_targ = get_ids(g_res, kind='desc'), get_ids(g_res, kind='targ')
        
    
        avl_desc = avl_desc
        avl_targ = g_res_desc.difference(avl_desc).union(avl_targ.difference(g_res_targ))
        
        avl_grph = [g for g in avl_graph
                    if len(g_nxt_mods.intersection(set(g))) > 0]
    
    g_res = prune(g_res)
    
    return g_res

So what we need are modifications to the mrai_algorithm that allow it to be more easily used as a component. Or, alternatively, split it up.